# Data Engineering 2: Lab 06 - Solution
---------------

In [0]:
# Import graphframes into the libaries of your cluster (take the jar of the zip archive)

from graphframes import *
from pyspark.sql.functions import desc
from seaborn.external.docscrape import header

#TODO: replace these paths with your csv upload paths

celebrities = spark.read.option("inferSchema", "true").csv("path", sep=",", header=True)
celebrities.printSchema()

followers = spark.read.option("inferSchema", "true").csv("path", sep=",", header=True)
followers.printSchema()

In [ ]:
# TODO: print 10 entries of the celebrities

celebrities.toPandas().head(10)

# TODO: print 3 entries of the Followers == edge table

followers.toPandas().head(3)

In [0]:
#TODO: build graph

celebrities_follower = GraphFrame(celebrities, followers)
print(celebrities_follower.show())

In [0]:
#TODO: print the age of the youngest celebrity

display(celebrities_follower.vertices.groupBy().min("age"))

#TODO: print the age of the oldest celebrity

oldest = celebrity_follower.vertices.groupBy().max("age")
display(oldest)

In [0]:
# Find out more info on the oldest person in the graph
# Note: we can use the collect() API to return the list of results (in this case, only 1) 
# so in order to get the actual value for maxAge we need to:

# TODO: 1. collect the results

results = oldest.collect()

# TODO: 2. get the first element in the list == a row

firstElem = results[0]

# TODO: 3. get the value of the field we need and convert it to a string. Note: it's better to address by name, but could also have done it by ID (ie "firstElem[0]")

maxAge = str(firstElem["max(age)"])

# TODO: 4. filter by age, where the age must be = the maximum age we just computed

display(celebrities_follower.vertices.filter("age=" + maxAge))

In [0]:
# TODO: print the average age of all celebrities

display(celebrities_follower.vertices.groupBy().avg("age"))

In [0]:
# TODO: print the number of incoming edges per vertex

display(celebrities_follower.inDegrees)

In [0]:
# TODO: print the number of outgoing edges per vertex

display(celebrities_follower.outDegrees)

In [0]:
# TODO: print the most influential celebrity will have the biggest number of followers
# first we need to retrieve this value

maxNumFollowers = celebrity_follower.inDegrees.groupBy().max("inDegree")
display(maxNumFollowers)

In [0]:
# TODO: Find out more info on the most influential in the graph

# we use the same trick as above to get the value of the id for the most influential celebrity
# the collect() API always returns a list
# so we need to take the first element (row) and then extract from it the id

# we use a filter to return the id of the celebrity with that number of followers

results = maxNumFollowers.collect()
firstElem = results[0]
maxNumFollowersValue = maxNumFollowers.collect()[0]["max(inDegree)"]
mostFollowedCelebrityID = celebrity_follower.inDegrees.filter('inDegree=' + str(maxNumFollowersValue))
display(mostFollowedCelebrityID)

In [0]:
# TODO: Get more information about the most followed celebrity. Join back (by ID) with the original celebrities dataframe and print the result.

mostFollowedCelebrity = celebrity_follower.vertices.join(mostFollowedCelebrityID, "id")
display(mostFollowedCelebrity)

In [0]:
# TODO: count the number of celebrities by profession and sort descending

countProfessions = celebrities_follower.vertices.groupBy("profession").count().sort(desc("count"))
display(countProfessions)

In [0]:
# TODO: print reciprocal relationships (like in the lecture)

# Kanten-DataFrame (wir nehmen an: src folgt dst)
edges = celebrities_follower.edges.select("src", "dst")

# Vertauschte Version: dst folgt src
reversed_edges = edges.selectExpr("dst as src", "src as dst")

# Schnittmenge (reziproke Beziehungen)
reciprocal = edges.intersect(reversed_edges)

# Optional: sortieren und anzeigen
reciprocal = reciprocal.sort(desc("src"), desc("dst"))
display(reciprocal)

In [0]:
# TODO: print the motifs of the query before and filter for person1 = Lady Gaga
filtered = reciprocal.filter("person1.Name = 'Lady Gaga'")
display(filtered)

In [0]:
# TODO: print the motifs of the query before and filter out all professions = singer of person2
